<a href="https://colab.research.google.com/github/Anurag28-03/CodeSoft/blob/main/REAL_TIME_CRICKET_SCORE_GENERATION_USING_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests streamlit matplotlib pandas

In [ ]:
import os
os.environ["CRICAPI_KEY"] = "https://api.cricapi.com/v1/currentMatches? apikey=...&offset=0"

In [ ]:
Live/current matches: https://api.cricapi.com/v1/currentMatches?apikey=...&offset=0
Matches list: https://api.cricapi.com/v1/matches?apikey=...&offset=0
Matches list: https://api.cricapi.com/v1/matches?apikey=...&offset=0

In [ ]:
pip install requests streamlit matplotlib pandas

In [ ]:
import os
os.environ["CRICAPI_KEY"] = "https://api.cricapi.com/v1/currentMatches?apikey=...&offset=0"

In [ ]:
START
  |
  v
Read CRICAPI_KEY from environment
  |
  |-- if missing -> show error -> STOP
  v
Create HTTP session with retries (build_session)
  |
  v
Fetch live matches (fetch_current_matches)
  |
  |-- if API error -> show error -> STOP
  v
Show match dropdown (selectbox)
  |
  v
Extract current innings score from selected match
  |
  v
Update history (overs, runs) in session_state
  |
  v
Plot Runs vs Overs graph (from history)
  |
  v
Predict final score using Linear Regression (if T20/ODI)
  |
  v
Fetch scorecard (optional) and show Top batters/bowlers
  |
  v
END (but dashboard keeps running)

In [ ]:
BASE_URL = "https://api.cricapi.com/v1"
API_KEY = os.getenv("CRICAPI_KEY", "").strip()

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# =========================
# HTTP SESSION (retries)
# =========================
def build_session(retries: int = 3, backoff: float = 0.6) -> requests.Session:
    s = requests.Session()
    retry = Retry(
        total=retries,
        connect=retries,
        read=retries,
        status=retries,
        backoff_factor=backoff,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

SESSION = build_session()

In [ ]:
import os
import time
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Tuple

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import pandas as pd
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output

# =========================
# CONFIG
# =========================
BASE_URL = "https://api.cricapi.com/v1"
API_KEY = os.getenv("CRICAPI_KEY", "").strip()
DEFAULT_TIMEOUT = 10

if not API_KEY:
    raise RuntimeError("CRICAPI_KEY not set. Please run: os.environ['CRICAPI_KEY'] = 'YOUR_KEY'")

# =========================
# HTTP SESSION (retries)
# =========================
def build_session(retries: int = 3, backoff: float = 0.6) -> requests.Session:
    s = requests.Session()
    retry = Retry(
        total=retries,
        connect=retries,
        read=retries,
        status=retries,
        backoff_factor=backoff,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=frozenset(["GET"]),
        raise_on_status=False,
    )
    adapter = HTTPAdapter(max_retries=retry)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

SESSION = build_session()

def cricapi_get(endpoint: str, params: Dict[str, Any], timeout: int = DEFAULT_TIMEOUT) -> Dict[str, Any]:
    """
    Safe API GET:
    - adds apikey
    - timeout + retries
    - returns dict; if error -> {'__error__': '...'}
    """
    url = f"{BASE_URL}/{endpoint.lstrip('/')}"
    params = dict(params)
    params["apikey"] = API_KEY

    try:
        r = SESSION.get(url, params=params, timeout=timeout)
    except requests.exceptions.Timeout:
        return {"__error__": "timeout"}
    except requests.exceptions.RequestException as e:
        return {"__error__": f"network: {e}"}

    if r.status_code != 200:
        return {"__error__": f"http_{r.status_code}", "__text__": r.text[:200]}

    try:
        return r.json()
    except ValueError:
        return {"__error__": "bad_json", "__text__": r.text[:200]}

# =========================
# API FUNCTIONS
# =========================
def fetch_current_matches(offset: int = 0) -> List[Dict[str, Any]]:
    data = cricapi_get("currentMatches", {"offset": offset})
    if data.get("__error__"):
        return [{"__error__": data["__error__"], "__raw__": data}]
    matches = data.get("data", [])
    return matches if isinstance(matches, list) else []

def fetch_scorecard(match_id: str) -> Dict[str, Any]:
    return cricapi_get("match_scorecard", {"offset": 0, "id": match_id})

# =========================
# PARSING / UTILITIES
# =========================
@dataclass
class InningsScore:
    runs: int
    wickets: int
    overs: float
    team: str = ""

def parse_overs(overs_val: Any) -> float:
    """
    "46.3" means 46 overs + 3 balls => 46 + 3/6 = 46.5
    """
    if overs_val is None:
        return 0.0
    try:
        if isinstance(overs_val, (int, float)):
            return float(overs_val)

        s = str(overs_val).strip()
        if not s:
            return 0.0
        if "." in s:
            o, b = s.split(".", 1)
            o_i = int(o)
            b_i = int(b)
            b_i = max(0, min(b_i, 5))
            return o_i + (b_i / 6.0)
        return float(s)
    except Exception:
        return 0.0

def extract_live_innings_from_current_match(m: Dict[str, Any]) -> Optional[InningsScore]:
    """
    currentMatches usually has a 'score' list:
    last item has r,w,o and sometimes inning/team
    """
    score_list = m.get("score", [])
    if not isinstance(score_list, list) or not score_list:
        return None

    last = score_list[-1] if isinstance(score_list[-1], dict) else None
    if not last:
        return None

    runs = int(last.get("r", 0) or 0)
    wickets = int(last.get("w", 0) or 0)
    overs = parse_overs(last.get("o"))
    team = str(last.get("inning", "") or last.get("team", "") or "")
    return InningsScore(runs=runs, wickets=wickets, overs=overs, team=team)

def infer_total_overs(match_obj: Dict[str, Any]) -> int:
    """
    Simple heuristic:
    - T20 => 20
    - ODI => 50
    - Test => 0 (skip prediction)
    """
    mt = (match_obj.get("matchType") or match_obj.get("type") or "").lower()
    name = (match_obj.get("name") or "").lower()
    text = f"{mt} {name}"

    if "t20" in text:
        return 20
    if "odi" in text or "one day" in text:
        return 50
    if "test" in text:
        return 0
    return 20  # default

def calculate_crr(runs: int, overs: float) -> float:
    return (runs / overs) if overs > 0 else 0.0

def linear_regression_predict(history: List[Tuple[float, int]], target_overs: int) -> Optional[float]:
    """
    Least squares regression: runs = a*overs + b
    history: [(overs, runs), ...]
    """
    if target_overs <= 0 or len(history) < 2:
        return None

    xs = [h[0] for h in history]
    ys = [h[1] for h in history]

    if max(xs) - min(xs) < 1e-6:
        return None

    n = len(xs)
    x_mean = sum(xs) / n
    y_mean = sum(ys) / n

    num = sum((x - x_mean) * (y - y_mean) for x, y in zip(xs, ys))
    den = sum((x - x_mean) ** 2 for x in xs)
    if abs(den) < 1e-9:
        return None

    a = num / den
    b = y_mean - a * x_mean
    pred = a * target_overs + b
    return max(0.0, pred)

def parse_scorecard_top_players(scorecard_json: Dict[str, Any]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Best-effort parsing for top batters/bowlers from scorecard.
    """
    data = scorecard_json.get("data", {}) if isinstance(scorecard_json.get("data"), dict) else {}
    scorecard = data.get("scorecard")

    bat_rows = []
    bowl_rows = []

    if isinstance(scorecard, list) and scorecard:
        inn = scorecard[-1] if isinstance(scorecard[-1], dict) else {}
        batting = inn.get("batting", []) or inn.get("batsmen", []) or []
        bowling = inn.get("bowling", []) or inn.get("bowlers", []) or []

        if isinstance(batting, list):
            for p in batting:
                if isinstance(p, dict):
                    bat_rows.append({
                        "name": p.get("name") or p.get("batsman") or p.get("player") or "",
                        "runs": p.get("r") or p.get("runs") or 0,
                        "balls": p.get("b") or p.get("balls") or 0,
                        "sr": p.get("sr") or p.get("strikeRate") or None,
                        "out": p.get("out") or p.get("dismissal") or "",
                    })

        if isinstance(bowling, list):
            for p in bowling:
                if isinstance(p, dict):
                    bowl_rows.append({
                        "name": p.get("name") or p.get("bowler") or p.get("player") or "",
                        "overs": p.get("o") or p.get("overs") or None,
                        "runs": p.get("r") or p.get("runs") or None,
                        "wkts": p.get("w") or p.get("wickets") or None,
                        "econ": p.get("econ") or p.get("economy") or None,
                    })

    bat_df = pd.DataFrame(bat_rows)
    bowl_df = pd.DataFrame(bowl_rows)

    if not bat_df.empty and "runs" in bat_df.columns:
        bat_df["runs"] = pd.to_numeric(bat_df["runs"], errors="coerce").fillna(0)
        bat_df = bat_df.sort_values("runs", ascending=False).head(7)

    if not bowl_df.empty and "wkts" in bowl_df.columns:
        bowl_df["wkts"] = pd.to_numeric(bowl_df["wkts"], errors="coerce").fillna(0)
        bowl_df = bowl_df.sort_values("wkts", ascending=False).head(7)

    return bat_df, bowl_df

# =========================
# COLAB DASHBOARD (widgets)
# =========================
history_by_match: Dict[str, List[Tuple[float, int]]] = {}

# UI Widgets
refresh_btn = widgets.Button(description="🔄 Refresh matches", button_style="info")
update_btn = widgets.Button(description="📊 Update selected match", button_style="success")
auto_refresh_chk = widgets.Checkbox(value=False, description="Auto refresh (30s)")
show_scorecard_chk = widgets.Checkbox(value=True, description="Show scorecard")
keep_hist_slider = widgets.IntSlider(value=60, min=10, max=200, step=10, description="Keep history")

matches_dropdown = widgets.Dropdown(options=[], description="Match:", layout=widgets.Layout(width="95%"))
status_out = widgets.Output()
main_out = widgets.Output()

def build_match_label(m: Dict[str, Any]) -> Optional[Tuple[str, str]]:
    """
    returns (label, match_id)
    """
    mid = m.get("id") or m.get("match_id") or m.get("unique_id")
    if not mid:
        return None
    name = m.get("name") or ""
    status = m.get("status") or m.get("matchStatus") or ""
    t1 = m.get("team1") or m.get("t1") or ""
    t2 = m.get("team2") or m.get("t2") or ""
    label_name = name.strip() if name else f"{t1} vs {t2}"
    label = f"{label_name} | {status} | id={mid}"
    return label, str(mid)

def load_matches():
    with status_out:
        clear_output()
        print("Fetching live matches...")

    matches = fetch_current_matches(0)
    if matches and isinstance(matches[0], dict) and matches[0].get("__error__"):
        with status_out:
            clear_output()
            print("API Error:", matches[0].get("__error__"))
        matches_dropdown.options = []
        return

    label_to_match = {}
    options = []
    for m in matches:
        built = build_match_label(m)
        if not built:
            continue
        label, mid = built
        options.append(label)
        label_to_match[label] = (mid, m)

    if not options:
        with status_out:
            clear_output()
            print("No live matches found right now (or API limit).")
        matches_dropdown.options = []
        return

    # Keep previous selection if possible
    prev = matches_dropdown.value
    matches_dropdown.options = options

    if prev in options:
        matches_dropdown.value = prev
    else:
        matches_dropdown.value = options[0]

    # Store map in widget object for access
    matches_dropdown._label_to_match = label_to_match

    with status_out:
        clear_output()
        print(f"Loaded {len(options)} live matches. Select one and click 'Update selected match'.")

def update_selected_match():
    if not matches_dropdown.options:
        with main_out:
            clear_output()
            print("No matches to show. Click Refresh matches first.")
        return

    selected_label = matches_dropdown.value
    if selected_label is None:
        with main_out:
            clear_output()
            print("Please select a match.")
        return

    label_to_match = getattr(matches_dropdown, "_label_to_match", {})
    if selected_label not in label_to_match:
        with main_out:
            clear_output()
            print("Selection invalid (options refreshed). Please re-select.")
        return

    match_id, match_obj = label_to_match[selected_label]

    # Extract live innings
    inning = extract_live_innings_from_current_match(match_obj)
    total_overs = infer_total_overs(match_obj)

    with main_out:
        clear_output()

        print("===================================================")
        print("🏏 Selected Match:", selected_label)
        print("===================================================")

        if not inning:
            print("No live innings score found in current match data.")
            return

        crr = calculate_crr(inning.runs, inning.overs)
        print(f"Score: {inning.runs}/{inning.wickets}")
        print(f"Overs: {inning.overs:.2f}")
        print(f"CRR  : {crr:.2f}")
        print(f"Format Overs (for prediction): {total_overs if total_overs>0 else 'N/A'}")

        # Update history
        hist = history_by_match.get(match_id, [])
        if len(hist) == 0 or inning.overs > hist[-1][0] + 1e-6:
            hist.append((inning.overs, inning.runs))

        # Trim history
        keep_n = int(keep_hist_slider.value)
        if len(hist) > keep_n:
            hist = hist[-keep_n:]
        history_by_match[match_id] = hist

        # Plot graph
        if len(hist) >= 2:
            xs = [h[0] for h in hist]
            ys = [h[1] for h in hist]

            plt.figure(figsize=(8,4))
            plt.plot(xs, ys, marker="o")
            plt.xlabel("Overs")
            plt.ylabel("Cumulative Runs")
            plt.title("Runs vs Overs (from your refresh history)")
            plt.grid(True)
            plt.show()

            pred = linear_regression_predict(hist, total_overs) if total_overs > 0 else None
            if pred is not None:
                print(f"📈 Predicted Final Score (Linear Regression): ~ {pred:.0f} runs at {total_overs} overs")
            else:
                print("Prediction not available (need 2+ points and fixed overs format like T20/ODI).")
        else:
            print("Graph & prediction will appear after 2+ updates (overs must increase).")

        # Scorecard tables
        if show_scorecard_chk.value:
            print("\nFetching scorecard...")
            sc = fetch_scorecard(match_id)
            if sc.get("__error__"):
                print("Scorecard error:", sc.get("__error__"))
                return

            bat_df, bowl_df = parse_scorecard_top_players(sc)

            print("\n🏏 Top Batters (latest innings)")
            display(bat_df if not bat_df.empty else pd.DataFrame({"note": ["No batting table parsed for this match."]}))

            print("\n🎯 Top Bowlers (latest innings)")
            display(bowl_df if not bowl_df.empty else pd.DataFrame({"note": ["No bowling table parsed for this match."]}))

def on_refresh_clicked(_):
    load_matches()

def on_update_clicked(_):
    update_selected_match()

refresh_btn.on_click(on_refresh_clicked)
update_btn.on_click(on_update_clicked)

controls = widgets.VBox([
    widgets.HBox([refresh_btn, update_btn]),
    matches_dropdown,
    widgets.HBox([auto_refresh_chk, show_scorecard_chk]),
    keep_hist_slider,
    status_out,
    main_out
])

display(controls)

# initial load
load_matches()

# Auto-refresh loop (optional)
def auto_loop():
    while auto_refresh_chk.value:
        update_selected_match()
        time.sleep(30)

auto_btn = widgets.Button(description="▶ Start Auto Refresh Loop", button_style="warning")
stop_btn = widgets.Button(description="⏹ Stop Auto Refresh", button_style="danger")
auto_out = widgets.Output()

def start_auto(_):
    with auto_out:
        clear_output()
        print("Auto refresh started (every 30s). To stop, uncheck auto refresh or click Stop.")
    auto_refresh_chk.value = True
    auto_loop()

def stop_auto(_):
    auto_refresh_chk.value = False
    with auto_out:
        clear_output()
        print("Auto refresh stopped.")

auto_btn.on_click(start_auto)
stop_btn.on_click(stop_auto)

display(widgets.HBox([auto_btn, stop_btn]), auto_out)